In [7]:
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

#Training에 사용할 Data
x_data = [[1, 2, 1],
          [1, 3, 2],
          [1, 3, 4],
          [1, 5, 5],
          [1, 7, 5],
          [1, 2, 5],
          [1, 6, 6],
          [1, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#Testing에서만 사용할 Data
x_test = [[2, 1, 1],
          [3, 1, 2],
          [3, 3, 4]]
y_test = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1]]

In [10]:
X = tf.placeholder("float", [None,3])
Y = tf.placeholder("float", [None,3])
W = tf.Variable(tf.random_normal([3,3]))
b = tf.Variable(tf.random_normal([3]))

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

# 값이 너무 작거나 크면 잘 나오지 않기 때문에, Accuracy를 잘 살피면서 보아야 함
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Correct prediction Test model
prediction = tf.arg_max(hypothesis, 1)
is_correct = tf.equal(prediction, tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(201):
        cost_val, W_val, _ = sess.run(
            [cost, W, optimizer], feed_dict={X: x_data, Y: y_data})
        print(step, cost_val, W_val)

    # predict
    print("Prediction:", sess.run(prediction, feed_dict={X: x_test}))
    # Calculate the accuracy
    print("Accuracy: ", sess.run(accuracy, feed_dict={X: x_test, Y: y_test}))

0 7.125784 [[ 1.7783344  -2.1305397  -1.2754714 ]
 [ 0.30258822  1.3275073  -0.25387156]
 [ 1.4680567   0.6731614  -0.25261587]]
1 4.722013 [[ 1.7128862  -2.102574   -1.2379888 ]
 [ 0.08195183  1.44818    -0.15390787]
 [ 1.237176    0.8165612  -0.16513503]]
2 3.9950583 [[ 1.6933953  -2.1205335  -1.2005386 ]
 [ 0.11847697  1.3117613  -0.05401424]
 [ 1.2461396   0.72015595 -0.07769342]]
3 3.583568 [[ 1.6577438  -2.1223044  -1.1631163 ]
 [ 0.07315755  1.2572502   0.04581624]
 [ 1.1757228   0.7031684   0.009711  ]]
4 3.25736 [[ 1.6260811  -2.1279938  -1.1257644 ]
 [ 0.04876613  1.1819736   0.14548431]
 [ 1.1256571   0.6659307   0.09701455]]
5 2.9336514 [[ 1.5925612  -2.1316996  -1.0885388 ]
 [ 0.01354228  1.1178374   0.24484429]
 [ 1.0653979   0.6390931   0.18411136]]
6 2.6129048 [[ 1.5603489  -2.1364455  -1.0515803 ]
 [-0.01580273  1.0485276   0.34349915]
 [ 1.011002    0.6069135   0.27068692]]
7 2.2978594 [[ 1.5280366  -2.1405108  -1.0152028 ]
 [-0.04742536  0.9832659   0.44038355]
 [ 0.

In [18]:
import numpy as np

#원래 DataSet인 xy
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

#Data가 Regularization이 필요하면 원래 데이터인 xy에다가 training_xy = MinMaxScaler(xy) 처럼 넣어서 스케일링 또한 가능 
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)

xy = MinMaxScaler(xy)

#5차원 데이터셋 xy에서, 앞의 4개는 x변수 뒤의 1개는 y변수
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([4, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(101):
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

0 Cost:  1.5980148 
Prediction:
 [[-1.3171978 ]
 [-0.60374963]
 [-0.5730814 ]
 [-0.5377667 ]
 [-0.6348387 ]
 [-0.71919584]
 [-0.27373335]
 [-0.23893516]]
1 Cost:  1.5979009 
Prediction:
 [[-1.3171339 ]
 [-0.6036898 ]
 [-0.5730313 ]
 [-0.5377281 ]
 [-0.6347927 ]
 [-0.7191515 ]
 [-0.27370456]
 [-0.23890755]]
2 Cost:  1.5977869 
Prediction:
 [[-1.3170699 ]
 [-0.6036298 ]
 [-0.5729815 ]
 [-0.5376893 ]
 [-0.63474655]
 [-0.71910715]
 [-0.27367574]
 [-0.23887995]]
3 Cost:  1.5976728 
Prediction:
 [[-1.317006  ]
 [-0.6035701 ]
 [-0.5729315 ]
 [-0.53765047]
 [-0.63470054]
 [-0.7190628 ]
 [-0.27364695]
 [-0.23885232]]
4 Cost:  1.5975585 
Prediction:
 [[-1.316942  ]
 [-0.60351026]
 [-0.5728816 ]
 [-0.5376117 ]
 [-0.6346544 ]
 [-0.71901846]
 [-0.27361816]
 [-0.23882473]]
5 Cost:  1.5974445 
Prediction:
 [[-1.3168781 ]
 [-0.6034504 ]
 [-0.57283163]
 [-0.53757286]
 [-0.6346084 ]
 [-0.7189741 ]
 [-0.27358937]
 [-0.23879711]]
6 Cost:  1.5973303 
Prediction:
 [[-1.3168141 ]
 [-0.6033906 ]
 [-0.57278156